<p align="center"><img src="https://github.com/wjf5203/VNext/raw/main/assets/VNext.png" width="300"/></p>

- VNext is a **Next**-generation **V**ideo instance recognition framework on top of [Detectron2](https://github.com/facebookresearch/detectron2). 
- Currently it provides advanced online and offline video instance segmentation algorithms.

## IDOL

### Introduction

- In recent years, video instance segmentation (VIS) has been largely advanced by offline models, while online models are usually inferior to the contemporaneous offline models by over 10 AP, which is a huge drawback.

- By dissecting current online models and offline models, we demonstrate that the main cause of the performance gap is the error-prone association and  propose IDOL, which outperforms all online and offline methods on three benchmarks. 

- IDOL won first place in the video instance segmentation track of the 4th Large-scale Video Object Segmentation Challenge (CVPR2022).

# Installation

***

First, ensure the runtime type is set to GPU. Then clone the repository locally:

In [1]:
!rm -r /content/*

In [2]:
!git clone https://github.com/wjf5203/VNext.git
%cd VNext

Cloning into 'VNext'...
remote: Enumerating objects: 938, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 938 (delta 79), reused 64 (delta 64), pack-reused 813
Receiving objects: 100% (938/938), 54.85 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (210/210), done.
/content/VNext


Install dependencies and pycocotools for VIS:

In [3]:
!pip install -r requirements.txt
!pip install -e .
!pip install shapely==1.7.1
!pip install git+https://github.com/youtubevos/cocoapi.git\#"egg=pycocotools&subdirectory=PythonAPI" # not sure if this works

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 24.4 MB/s 
     |████████████████████████████████| 182 kB 77.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/VNext
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 154 kB 75.9 MB/s 
     |████████████████████████████████| 130 kB 65.2 MB/s 
     |████████████████████████████████| 117 kB 68.4 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=fd5e9d30880332eedc437a687b46937afcb3fcee506db7349cc659a9772e0358
  Stored in directory: /root/.cache/pip/wheels/b8/79/07/c0e9367f5b5ea325e246bd73651e8af175fabbef943043b1cc
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 31.6 MB/s 
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.0
    Uninstalling shapely-2.0.0:
      Successfully uninstalled shapely-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/youtubevos/cocoapi.git to /tmp/pip-install-kay_f9tm/pycocotools_730e969616e343de9b7add8b0e573642
  Running command git clone -q https://github.com/youtubevos/cocoapi.git /tmp/pip-install-kay_f9tm/pycocotools_730e969616e343de9b7add8b0e573642
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-linux_x86_64.whl size=314571 sha256=119e7570088ca06d7fa9f28fc46d3857c7827a9381e9672ccee8e0daeb66006d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uhqrbk4i/wheels/0c/d8/20/6df2e08a2f4205e206c7b899cbbebadaabf08e0123e884572f
Successfully built pyco

Compiling Deformable DETR CUDA operators:

In [4]:
%cd projects/IDOL/idol/models/ops/
!sh make.sh

/content/VNext/projects/IDOL/idol/models/ops
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.8
creating build/lib.linux-x86_64-3.8/functions
copying functions/__init__.py -> build/lib.linux-x86_64-3.8/functions
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-3.8/functions
creating build/lib.linux-x86_64-3.8/modules
copying modules/ms_deform_attn.py -> build/lib.linux-x86_64-3.8/modules
copying modules/__init__.py -> build/lib.linux-x86_64-3.8/modules
running build_ext
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:387: UserWarning: The detected CUDA version (11.2) has a minor version mismatch with the version that was used to compile PyTorch (11

# Data Preparation

***

Download and extract 2019 version of YoutubeVIS train and val images with annotations from CodeLab or YouTubeVIS, download OVIS and COCO 2017 datasets.

In [5]:
# download validation set

# designed for Google Colab
%cd /content/
for i in range(15):
  i = i + 1
  url = f"https://altitude.otago.ac.nz/jeshanks/valid/-/raw/main/valid.zip.{i:03}"
  !wget $url # shows error but works 

# concatenate all part files to one zip
!cat /content/valid.zip* > /content/valid.zip
!rm /content/valid.zip.*

# repair combination
!zip -FF /content/valid.zip --out /content/valid-full.zip
!rm /content/valid.zip

# unzip combination
!apt-get install -y unzip
!unzip /content/valid-full.zip
!rm /content/valid-full.zip

# download coco pretrain data
!wget https://jeshanks.cspages.otago.ac.nz/vrvideos/cocopretrain_R50.pth

# create appropriate structure directory
!mkdir /content/VNext/datasets/ytvis_2021
!mkdir /content/VNext/datasets/ytvis_2021/annotations

Streaming output truncated to the last 5000 lines.
  inflating: valid/JPEGImages/a79cf29daa/00150.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00155.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00160.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00165.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00170.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00175.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00180.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00185.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00190.jpg  
  inflating: valid/JPEGImages/a79cf29daa/00195.jpg  
   creating: valid/JPEGImages/a7c254950c/
  inflating: valid/JPEGImages/a7c254950c/00000.jpg  
  inflating: valid/JPEGImages/a7c254950c/00005.jpg  
  inflating: valid/JPEGImages/a7c254950c/00010.jpg  
  inflating: valid/JPEGImages/a7c254950c/00015.jpg  
  inflating: valid/JPEGImages/a7c254950c/00020.jpg  
  inflating: valid/JPEGImages/a7c254950c/00025.jpg  
  inflating: valid/JPEGImages/a7c254950c/00030.jpg  
  infl

In [6]:
!cp /content/valid/instances.json /content/VNext/datasets/ytvis_2021/annotations/instances_val_sub.json

In [7]:
!mv /content/valid /content/VNext/datasets/ytvis_2021/val

Extract YouTube-VIS 2019, OVIS, COCO 2017 datasets, we expect the directory structure to be the following:

```
VNext
├── datasets
│   ├──ytvis_2019
│   ├──ovis 
│   ├──coco 
...
ytvis_2019
├── train
├── val
├── annotations
│   ├── instances_train_sub.json
│   ├── instances_val_sub.json
...
ovis
├── train
├── valid
├── annotations_train.json
├── annotations_valid.jso
...
coco
├── train2017
├── val2017
├── annotations
│   ├── instances_train2017.json
│   ├── instances_val2017.json
```

# Inference & Evaluation

***

Evaluating on YouTube-VIS 2021:


In [8]:
%cd /content/VNext
!python3 /content/VNext/projects/IDOL/train_net.py --help

/content/VNext
usage: train_net.py
       [-h]
       [--config-file FILE]
       [--resume]
       [--eval-only]
       [--num-gpus NUM_GPUS]
       [--num-machines NUM_MACHINES]
       [--machine-rank MACHINE_RANK]
       [--dist-url DIST_URL]
       ...

positional arguments:
  opts
    Modify
    config
    options at
    the end of
    the
    command.
    For Yacs
    configs,
    use space-
    separated
    "PATH.KEY
    VALUE"
    pairs. For
    python-
    based
    LazyConfig,
    use "path.k
    ey=value".

optional arguments:
  -h, --help
    show this
    help
    message and
    exit
  --config-file FILE
    path to
    config file
  --resume
    Whether to
    attempt to
    resume from
    the
    checkpoint
    directory.
    See documen
    tation of `
    DefaultTrai
    ner.resume_
    or_load()`
    for what it
    means.
  --eval-only
    perform
    evaluation
    only
  --num-gpus NUM_GPUS
    number of
    gpus *per
    machine*
  --num-machines NUM_MACHINES
 

In [9]:
%cd /content/VNext
!python3 /content/VNext/projects/IDOL/train_net.py --config-file /content/VNext/projects/IDOL/configs/ytvis21_r50.yaml --num-gpus 1 --eval-only MODEL.WEIGHTS /content/cocopretrain_R50.pth

/content/VNext
Command Line Args: Namespace(config_file='/content/VNext/projects/IDOL/configs/ytvis21_r50.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', '/content/cocopretrain_R50.pth'], resume=False)
[12/23 01:09:17 detectron2]: Rank of current process: 0. World size: 1
[12/23 01:09:18 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.8.16 (default, Dec  7 2022, 01:12:13) [GCC 7.5.0]
numpy                   1.21.6
detectron2              0.6 @/content/VNext/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.2
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.13.0+cu116 @/usr/local/lib/python3.8/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla T4 (arch=7.5)
Driver ver

**WARNING: ** if you have a ModuleNotFoundError then make.sh failed.

In [15]:
%cd /content
!zip -r /content/IDOL_YTVIS21_R50.zip /content/VNext/IDOL_YTVIS21_R50

/content
updating: content/VNext/IDOL_YTVIS21_R50/ (stored 0%)
updating: content/VNext/IDOL_YTVIS21_R50/config.yaml (deflated 65%)
updating: content/VNext/IDOL_YTVIS21_R50/inference/ (stored 0%)
updating: content/VNext/IDOL_YTVIS21_R50/log.txt (deflated 89%)


In [44]:
from google.colab import files
files.download('/content/IDOL_YTVIS21_R50.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
!zip -r /content/VNext.zip /content/VNext

Streaming output truncated to the last 5000 lines.
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00035.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00120.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00020.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00135.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00140.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00110.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00100.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00040.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00050.jpg (deflated 1%)
  adding: content/VNext/datasets/ytvis_2021/val/JPEGImages/a6569911cb/00030.jpg (deflated 1%)
  adding:

In [43]:
files.download('/content/VNext.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
!cp /content/VNext.zip /content/drive/MyDrive

## Experimental

In [14]:
%cd /content/VNext
!python3 /content/VNext/projects/IDOL/train_net.py --num-gpus 1 --eval-only MODEL.WEIGHTS /content/cocopretrain_R50.pth

/content/VNext
Command Line Args: Namespace(config_file='', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', '/content/cocopretrain_R50.pth'], resume=False)
Traceback (most recent call last):
  File "/content/VNext/projects/IDOL/train_net.py", line 174, in <module>
    launch(
  File "/content/VNext/detectron2/engine/launch.py", line 82, in launch
    main_func(*args)
  File "/content/VNext/projects/IDOL/train_net.py", line 157, in main
    cfg = setup(args)
  File "/content/VNext/projects/IDOL/train_net.py", line 149, in setup
    cfg.merge_from_file(args.config_file)
  File "/content/VNext/detectron2/config/config.py", line 45, in merge_from_file
    assert PathManager.isfile(cfg_filename), f"Config file '{cfg_filename}' does not exist!"
AssertionError: Config file '' does not exist!


the pipeline must have a config file

In [41]:
!rm /content/tokyostones.zip
!rm /content/*.png
!wget https://jeshanks.cspages.otago.ac.nz/vrvideos/tokyostones.zip
!unzip /content/tokyostones.zip
!rm /content/tokyostones.zip

rm: cannot remove '/content/tokyostones.zip': No such file or directory
--2022-12-23 03:27:47--  https://jeshanks.cspages.otago.ac.nz/vrvideos/tokyostones.zip
Resolving jeshanks.cspages.otago.ac.nz (jeshanks.cspages.otago.ac.nz)... 139.80.137.43
Connecting to jeshanks.cspages.otago.ac.nz (jeshanks.cspages.otago.ac.nz)|139.80.137.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 295112845 (281M) [application/zip]
Saving to: ‘tokyostones.zip’

tokyostones.zip     100%[===================>] 281.44M  10.2MB/s    in 20s     

2022-12-23 03:28:08 (13.8 MB/s) - ‘tokyostones.zip’ saved [295112845/295112845]

Archive:  /content/tokyostones.zip
 extracting: 000000.png              
 extracting: 000001.png              
 extracting: 000002.png              
 extracting: 000003.png              
 extracting: 000004.png              
 extracting: 000005.png              
 extracting: 000006.png              
 extracting: 000007.png              
 extracting: 000008.png   

In [42]:
# designed for Google Colab
%cd /content/
for i in range(15):
  i = i + 1
  url = f"https://altitude.otago.ac.nz/jeshanks/vrvideos/-/raw/main/valid.zip.{i:03}"
  !wget $url # shows error but works 

# concatenate all part files to one zip
!cat /content/valid.zip* > /content/valid.zip
!rm /content/valid.zip.*

# repair combination
!zip -FF /content/valid.zip --out /content/valid-full.zip
!rm /content/valid.zip

# unzip combination
!apt-get install -y unzip
!unzip /content/valid-full.zip
!rm /content/valid-full.zip

Streaming output truncated to the last 5000 lines.
 copying: valid/JPEGImages/a7c254950c/  (0 bytes)
 copying: valid/JPEGImages/a7c254950c/00000.jpg  (210085 bytes)
 copying: valid/JPEGImages/a7c254950c/00005.jpg  (245217 bytes)
 copying: valid/JPEGImages/a7c254950c/00010.jpg  (243771 bytes)
 copying: valid/JPEGImages/a7c254950c/00015.jpg  (240696 bytes)
 copying: valid/JPEGImages/a7c254950c/00020.jpg  (233284 bytes)
 copying: valid/JPEGImages/a7c254950c/00025.jpg  (226940 bytes)
 copying: valid/JPEGImages/a7c254950c/00030.jpg  (224274 bytes)
 copying: valid/JPEGImages/a7c254950c/00035.jpg  (225116 bytes)
 copying: valid/JPEGImages/a7c254950c/00040.jpg  (225178 bytes)
 copying: valid/JPEGImages/a7c254950c/00045.jpg  (224035 bytes)
 copying: valid/JPEGImages/a7c254950c/00050.jpg  (224427 bytes)
 copying: valid/JPEGImages/a7c254950c/00055.jpg  (222140 bytes)
 copying: valid/JPEGImages/a7c254950c/00060.jpg  (222369 bytes)
 copying: valid/JPEGImages/a7c254950c/00065.jpg  (221044 bytes)
 co